# Notebook 4: Algorithmic Fairness - Considering Different Definitions

Decision making within the United States criminal justice system relies heavily on risk assessment, which determines the potential risk that a released defendant will fail to appear in court or cause harm to the public. Judges use these assessments to decide if bail can be set or if a defendant should be detained before trial. While this is not new in the legal system, the use of risk scores determined by an algorithm are gaining prevalence and support. Proponents promote the use of risk scores to guide judges in their decision making, arguing that machine learning could lead to greater efficiency, accountability, and less biased decisions compared with human judgment ([Henry](https://theappeal.org/risk-assessment-explained/)). On the other hand, critical voices raise the concern that biases can creep into these algorithms at any point in the process, and that algorithms are often applied to the wrong situations ([Henry](https://theappeal.org/risk-assessment-explained/)). Further, they exacerbate the racism embedded deep within the criminal justice system by perpetuating inequalities found in historical data ([Henry](https://theappeal.org/risk-assessment-explained/)).

In the debate about the use of risk assessment algorithms, people have used data analysis to determine the extent to which these algorithms are fair to different groups of people. In this notebook, **you will explore some of the many definitions and metrics (different ways of operationalizing data to quantify those definitions) of fairness that you will apply to a prominent  risk assessment tool called COMPAS in an attempt to evaluate whether the algorithm is biased**.  Along the way, you will examine the arguments and analyses made by the company that created COMPAS and the critics of this risk assessment tool to gain a deeper understanding of the technical and societal interpretations and implications of fairness. 


**NOTE**: When we discuss bias in this notebook, we define it most generally as prejudice or an inclination in favor of one person, thing, or group compared to another. In the context of machine learning, bias is a “phenomenon that occurs when an algorithm produces results that are systemically prejudiced due to erroneous assumptions in the machine learning process” ([Rouse](https://searchenterpriseai.techtarget.com/definition/machine-learning-bias-algorithm-bias-or-AI-bias#:~:text=Machine%20learning%20bias%2C%20also%20sometimes,in%20the%20machine%20learning%20process)).

## Learning Objectives
After completing this notebook, you will be able to:
- Identify how algorithms exist as part of socio-technical systems - as in, they are technological tools that are created by and interact with humans in social institutions.
- Recognize that there are different definitions of fairness that can be applied to evaluate algorithms, and critically examine how these definitions are emerging from and support different sociotechnical imaginaries of justice. 
- Understand how definitions of fairness are co-produced with racist systems/institutions. 
- Recognize the limits and opportunities of technical solutions to problems of algorithmic bias, and what historical and institutional contexts need to be considered to aim for fairness in algorithm design and deployment.
- Consider what kinds of community knowledge and other professional expertise besides their own is necessary to think through socio-technical issues.
- Consider what issues that are relevant to the COMPAS ecosystem (i.e. criminal justice system, affected communities) but outside of the algorithm itself, need to be addressed to be able to create a more just system, with or without the algorithm.


## Table of Contents:
* [Part 0. COMPAS](#part-zero)
* [Part 1. The Coproduction of Justice and Data](#part-one)
* [Part 2. ProPublica's Perspective](#part-two)
* [Part 3. Northpointe's Perspective](#part-three)
* [Part 4. Yet Another Definition of Fairness](#part-four)
* [Part 5. Conclusion](#part-five)

#### Dependencies

Let's begin by importing the packages we need that will give us the appropriate tools and methods needed.

In [ ]:
import otter
generator = otter.Notebook()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np # Loads numerical methodss
import pandas as pd # This allows us to transform our data into tables (AKA DataFrames) 

from sklearn.metrics import roc_curve, roc_auc_score # Loads visualization tools for a ROC Curve
import matplotlib.pyplot as plt # Loads more visualization tools
import seaborn as sns # Loads more visualization tools

# These install a fairness package with the DisparateImpactRemover tool used in Part 4
!pip install aif360
!pip install fairlearn
!pip install BlackBoxAuditing

from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import BinaryLabelDataset

---

# Part 0. COMPAS: Why it was created and how it exists in the court system <a id="part-zero"></a>

COMPAS (Correctional Offender Management Profiling for Alternative Sanctions) is a commercial tool produced by the for-profit company Northpointe known as a recidivism risk assessment system. Tools like COMPAS are used **to predict the risk of future crimes for an individual who has entered the US criminal justice system by outputting a risk score from 1-10**. While COMPAS was initially intended to aid decisions made by probation officers on treatment and supervision of those who are incarcerated, Northpointe has since emphasized the scalability of the tool to “fit the needs of many different decision points'' including pre-screening assessments, pretrial release decisions (whether or not to hold an arrested individual in jail until their trial), and post-trial next steps for the defendant ([Northpointe](http://www.northpointeinc.com/files/downloads/FAQ_Document.pdf)). These algorithms are believed by many to hold the power to relieve the court system of unfair human bias from criminal justice officials.

**NOTE**: In machine learning, COMPAS is considered a ***model***. A model is composed of a dataset that has ***features*** (the columns in the data table) and an ***algorithm***, which is a procedure that predicts a target feature from other features in the dataset. For example, COMPAS uses an algorithm to predict the likelihood of recidivism with a risk score (target feature) by using information about the defendant such as age and criminal history (represented in the columns of our dataset). 


<!-- BEGIN QUESTION -->

### Question 0a
Explain 3 parties that are impacted by the COMPAS tool. In what ways are they impacted? (Can you think of impacts beyond those in the courtroom for at least one of your examples?)

<!--
BEGIN QUESTION
name: q0a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->


<!-- BEGIN QUESTION -->
### Question 0b
Based on your initial reading, what is one problem of the criminal justice system that the COMPAS tool could potentially alleviate? What is one potential problem that using the COMPAS algorithm could introduce? 
<!--
BEGIN QUESTION
name: q0b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

----

# Part 1: The Coproduction of Justice and Data <a id="part-one"></a>

## Question 1. Understanding the Methods of Data Collection

Before a risk score is determined for a defendant, they are asked to fill out a questionnaire with questions meant to help predict the defendant’s level of risk. Let’s take a look at this questionnaire to get a better understanding of what goes into determining a risk score.

[Here](https://www.documentcloud.org/documents/2702103-Sample-Risk-Assessment-COMPAS-CORE.html) is a link to a sample questionnaire.

<!-- BEGIN QUESTION -->

### Question 1a

What aspects of the questionnaire were particularly striking to you? 

<!--
BEGIN QUESTION
name: q1a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1b 

How is race embedded in the questionnaire? 

<!--
BEGIN QUESTION
name: q1b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1c

How is this questionnaire influenced by the justice system? How does it influence the justice system?

<!--
BEGIN QUESTION
name: q1c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

---

## Question 2. Understanding the Data

We will be using the data that was obtained and utilized by ProPublica in their own analysis of the COMPAS tool from Broward County public records of people who were scored between 2013 and 2014 ([ProPublica](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm)). In order to follow ProPublica's analysis, we remove any cases where the charge was not within 30 days of the score (ProPublica did this in order to match the COMPAS score with the correct criminal case). We are left with 6172 rows in the dataset.

In [ ]:
data = pd.read_csv('compas-scores-two-years.csv')
data = data.query('days_b_screening_arrest <= 30 & days_b_screening_arrest >= -30')
data

We are also able to filter out any information that was not used by ProPublica and select fields for severity of charge, number of priors, demographics, age, sex, compas scores, and whether each person was accused of a crime within two years.

In [ ]:
select_data = data[["age", "c_charge_degree", "race", "age_cat", "score_text", "sex", "priors_count", 
                    "days_b_screening_arrest", "decile_score", "is_recid", "two_year_recid", "c_jail_in", "c_jail_out"]]
select_data.head()

<!-- BEGIN QUESTION -->

### Question 2a

Explore the dataset. What does a row in the dataset represent? 

<!--
BEGIN QUESTION
name: q2a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

***Sensitive features*** are features within a dataset that are given special consideration and treatment for potential legal, social, or ethical reasons. Often, these features are recognized and protected by antidiscrimination or privacy laws. One example of a sensitive feature is age. 

<!-- BEGIN QUESTION -->

### Question 2b

Identify 2 sensitive features in the dataset that we have not already mentioned.

<!--
BEGIN QUESTION
name: q2b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2c 

Pick one of the sensitive features you have identified. Identify at least 2 features in the dataset that are proxies for that sensitive feature. 

<!--
BEGIN QUESTION
name: q2c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 2d
In data science, why is it important to give special consideration to these kinds of features? 

<!--
BEGIN QUESTION
name: q2d
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

----

# Part 2. ProPublica’s Perspective <a id="part-two"></a>

### Who is ProPublica?

ProPublica is a nonprofit organization that “produces investigative journalism with moral force” ([ProPublica](https://www.propublica.org/about/)). ProPublica was founded as a nonpartisan newsroom aiming to expose and question abuses of power, justice, and public trust, often by systems and institutions deeply ingrained in the US.

In 2016, ProPublica investigated the COMPAS algorithm to assess the accuracy of and potential racial bias within the tool, as it became more popular within the United States court system nationwide. In their analysis, ProPublica used data from defendants with risk scores from Broward County from 2013 to 2014 to test for statistical differences in outcomes for Black and white defendants, which ultimately highlighted racial disparities that exist within the algorithm. ProPublica came to the conclusion that COMPAS utilizes data from a criminal justice system with a history of racial injustices, thus continuing to disproportionately target and arrest Black people in comparison to their white counterparts. While the COMPAS algorithm treats unequal groups alike, which may appear neutral, ProPublica’s data analysis and reporting emphasized the bias against Black defendants and their communities that COMPAS produced from this line of thinking, a claim that Northpointe has disputed (as we will see later).

Let's retrace ProPublica's statistical analysis in order to better understand ProPublica's argument and engage with the metric of fairness that it uses. In order to mimic their analysis more closely, we will use ProPublica’s definitions of “high” and “low” scores: any score above a 4 is considered a high score, while 4 and below is considered a low score. 


## Question 3. Visualization of Disparity 

Let’s visualize how ProPublica began their investigation of racial disparity within the COMPAS risk assessment. The histogram below displays the differences between the risk scores of black and white defendants. (Recall: The histogram definition and exploration from notebook 1.)

In [ ]:
# Code to create a histogram for the risk scores for white defendants
white_data = select_data[select_data["race"] == "Caucasian"]
sns.distplot(white_data["decile_score"], hist=True, rug=False)

# Code to create a histogram for the risk scores for Black defendants
black_data = select_data[select_data["race"] == "African-American"]
sns.distplot(black_data["decile_score"], hist=True, rug=False); # This plot will layer over the previous plot

<!-- BEGIN QUESTION -->

### Question 3a

Is one racial group more likely to get a high risk score opposed to the other? If so, why? (Hint: connect this to your knowledge of the history of policing and institutionalized racism.)

<!--
BEGIN QUESTION
name: q3a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

----

## Question 4. FPR and FNR: Does COMPAS overpredict or underpredict across groups?

Recall that models use algorithms to predict a target feature (in this case the likelihood of recidivism). We can evaluate how well a model predicts these features by using ***goodness metrics***. Goodness metrics typically summarize discrepancies between actual values and predicted values. Metrics like these are quite important in data science and typically are used by data scientists to determine how accurate and effective an algorithm is at predicting its predetermined goal.  

In order to answer this question and understand the ways in which bias is present in the risk scores, ProPublica used the **False Positive Rate (FPR)** and **False Negative Rate (FNR)** as their metrics to understand and quantify fairness. In terms of COMPAS, the FPR tells us the percentage of defendants that do not reoffend even though they were predicted to reoffend by the algorithm (*the defendants that did not reoffend and received a “high” score*). On the other hand, the FNR tells us the percentage of defendants that do reoffend even though they were predicted to not reoffend by the algorithm (*the defendants that did reoffend and received a “low” score*). 

Below we have calculated the False Positive Rates and False Negative rates based on the data from the risk scores.  We did this by creating functions that calculate each metric. You do not need to understand the code, but feel free to go through it yourself! [Here](https://medium.com/datadriveninvestor/confusion-matric-tpr-fpr-fnr-tnr-precision-recall-f1-score-73efa162a25f) is more information on how to calculate metrics like FPR and FNR if you are interested.  


In [ ]:
def fpr(race_feature, data):
    # Function returns the False Positive Rate of scores for the specified race_feature
    
    subgroup = data[data["race"] == race_feature]
    did_not_recidivate = subgroup[subgroup["two_year_recid"] == 0]

    false_positives = did_not_recidivate[did_not_recidivate["decile_score"] > 4].shape[0]
    true_negatives = did_not_recidivate[did_not_recidivate["decile_score"] <= 4].shape[0]
    return false_positives / (false_positives + true_negatives)


def fnr(race_feature, data):
    # Function returns the False Negative Rate of scores for the specified race_feature
    
    subgroup = data[data["race"] == race_feature]
    recidivated = subgroup[subgroup["two_year_recid"] == 1]
    
    false_negatives = recidivated[recidivated["decile_score"] <= 4].shape[0]
    true_positives = recidivated[recidivated["decile_score"] > 4].shape[0]
    return false_negatives / (false_negatives + true_positives)


In [ ]:
# Apply the metrics to the dataset
print("FPR for Black defendants:", round(fpr("African-American", select_data), 5) * 100, "%")
print("FPR for white defendants:", round(fpr("Caucasian", select_data), 5) * 100, "%")
print("FNR for Black defendants:", round(fnr("African-American", select_data), 4) * 100, "%")
print("FNR for white defendants:", round(fnr("Caucasian", select_data), 5) * 100, "%")

<!-- BEGIN QUESTION -->

### Question 4a

What do you notice about these percentages? (Hint: Are they different across racial groups?)
<!--
BEGIN QUESTION
name: q4a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 4b

What can you conclude from these metrics about the *overprediction* of risk scores for black and white defendants? (Hint: Look back to the definitions of FPR and FNR in the paragraphs above.)
<!--
BEGIN QUESTION
name: q4b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 4c

What can you conclude from these metrics about the *underprediction* of risk scores for black and white defendants? 
<!--
BEGIN QUESTION
name: q4c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 4d 

What is the importance of overprediction and underprediction in regard to ProPublica’s analysis? How might these observations have real impacts on the defendants who receive scores?
<!--
BEGIN QUESTION
name: q4d
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

----
##  Question 5.

<!-- BEGIN QUESTION -->

### Question 5a (i)
What kinds of problems does ProPublica highlight in the COMPAS algorithm? 

<!--
BEGIN QUESTION
name: q5ai
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 5a (ii)

How would you describe ProPublica’s definition of fairness, after learning and utilizing the metrics they used?

<!--
BEGIN QUESTION
name: q5aii
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 5b 

Why did ProPublica choose to investigate *racial* bias rather than a different sensitive feature? (Hint: think about how ProPublica’s conclusions reflect the racial disparities in our current criminal justice system.)

<!--
BEGIN QUESTION
name: q5b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 5c
What is ProPublica’s agenda as an investigative journalism organization? How do we see this in their analysis and conclusions?

<!--
BEGIN QUESTION
name: q5c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->


<!-- BEGIN QUESTION -->

### Question 5d

How do ProPublica’s conclusions reflect the racial disparities in our current criminal justice system? Why?

<!--
BEGIN QUESTION
name: q5d
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

We mentioned earlier that Northpointe disagreed with ProPublica's argument that the COMPAS algorithm is racially biased. Now that we’ve analyzed ProPublica’s perspective and seen the way in which they define and operationalize the concept of fairness, let’s move on to Northpointe’s.

---
# Part 3. Northpointe's Perspective <a id="part-three"></a>

### Who is Northpointe? (acquired by equivant)

Northpointe (merged with two other companies to create equivant in 2017) is a for-profit computer software company that aims to advance justice by informing and instilling confidence in decision makers at every stage of the criminal justice system ([equivant](https://www.equivant.com/)).  In addition to operating and continuing to develop COMPAS, *equivant* has developed a variety of technologies for use in court case management, attorney case management, inmate classification, and risk/needs assessment strategies.  

In the wake of criticism from ProPublica and other researchers alike, Northpointe produced a [detailed response](http://go.volarisgroup.com/rs/430-MBX-989/images/ProPublica_Commentary_Final_070616.pdf) to ProPublica’s allegations, claiming that these critiques of their tool utilized the wrong type of classification statistics in their analysis and portrayed the tool incorrectly. The company provided their own analysis of the COMPAS algorithm by using different statistical methods and responding individually to each of ProPublica’s claims of racial bias against Black defendants. 

Upon examining their tool’s fairness through a metric called *accuracy equity* (a metric that was left out of ProPublica’s analysis), as well as the fact that the model was not trained with a race feature, Northpointe concluded that their algorithm treats all people and specified groups equally, and therefore does not exhibit signs of bias or inequality for specified groups. Now, let’s take a look at how Northpointe supported this argument.


---
## Question 6. Sensitive Features: How can a model be racially biased without a race feature?

Northpointe made it very clear that they did not utilize any race feature (a column that informs the algorithm of the race of each defendant) in their dataset when they were training their algorithm. Northpointe decided that as a result of neglecting this race feature, the COMPAS tool would not take race into account when making decisions for defendants and therefore cannot be biased against certain races. 

<!-- BEGIN QUESTION -->
### Question 6a

Think back to our discussion about sensitive features and proxies. Can you think of any way race is *still* embedded in the algorithm, even though it does not use a race feature to make decisions?

<!--
BEGIN QUESTION
name: q6a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 6b

Why might neglecting to consider race in the context of the US criminal justice system be harmful or problematic? 

<!--
BEGIN QUESTION
name: q6b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 6c

Can data scientists’ algorithms be absolved of racial bias if they decide to not take race into account within their models? Explain why or why not. 

<!--
BEGIN QUESTION
name: q6c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

---

## Question 7: Accuracy Equity: Is each group being discriminated against equally?

Instead of analyzing and comparing the model errors FNR and FPR (the metrics you calculated in the previous ProPublica section), Northpointe utilized complement of FNR, known as the TPR (or what is often referred to as *Sensitivity*), paired with the FPR in a visualization called the ROC Curve. Once again, the TPR is a new way to measure the “goodness” of a model, similar to FNR and FPR. Northpointe made use of the *ROC Curve*, which we describe below, to prove what they refer to as ***Accuracy Equity***. Accuracy equity, according to [Northpointe](http://go.volarisgroup.com/rs/430-MBX-989/images/ProPublica_Commentary_Final_070616.pdf), is exhibited in the model “if it can discriminate recidivists and nonrecidivists equally well for two different groups such as blacks and whites.” In order to connect these two ideas, we first need to learn what this curve does and how it is used to understand fairness. 

In data science, we can use a visualization called the **Receiver Operating Characteristic (ROC) curve** to interpret ***how much a model is capable of distinguishing between classes***. We do this by plotting the curve and deriving the **Area Under the Curve (AUC)**. The higher the AUC, the better the model is at predicting our target variable. Northpointe utilized this method by plotting metrics we have already calculated: the FPR on the x-axis and TPR on the y-axis, and then calculating and comparing the **AUC** for the subgroup of data with white defendants and the subgroup of data with Black defendants.

Below is the code to create this visualization. You do not need to understand the code itself, but feel free to go through it yourself! 

In [ ]:
# Calculate FPR and FNR from metrics package - Black defendants
black_def = select_data[select_data["race"] == "African-American"]
# True values (if the defendants reoffended)
y1 = black_def['two_year_recid']
# Predicted values from COMPAS tool (risk scores)
pred1 = black_def["decile_score"].replace([1, 2, 3, 4], 0).replace([5, 6, 7, 8, 9, 10], 1)
fpr_black, tpr_black, threshold = roc_curve(y1, pred1)

# Calculate FPR and FNR from metrics package - White defendants
white_def = select_data[select_data["race"] == "Caucasian"]
# True values (if the defendants reoffended)
y2 = white_def['two_year_recid']
# Predicted values from COMPAS tool (risk scores)
pred2 = white_def["decile_score"].replace([1, 2, 3, 4], 0).replace([5, 6, 7, 8, 9, 10], 1)
fpr_white, tpr_white, threshold = roc_curve(y2, pred2)

# Set up the plot for the ROC curve
plt.subplots(1, figsize=(10,10))
plt.title('ROC - Black Defendents')
plt.plot(fpr_black, tpr_black)
plt.plot(fpr_white, tpr_white)
plt.plot([0, 1], ls="--")
plt.ylabel('Sensitivity')
plt.xlabel('1 - Specificity')
plt.show()


In [ ]:
# Calculate and print the AUC
print("AUC for Black defendants:", roc_auc_score(y1, pred1))
print("AUC for white defendants:", roc_auc_score(y2, pred2))

<!-- BEGIN QUESTION -->
### Question 7a (i). 

What do you notice from the ROC curve and the AUC calculation? Please list at least 2 observations.
<!--
BEGIN QUESTION
name: q7ai
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 7a (ii). 
What could Northpointe take away from this visualization to prove their point? (Hint: Is each racial group being discriminated against equally? Can you distinguish between racial groups or are they more or less the same?) 

Recall: the AUC helps us determine by *how much a model is capable of distinguishing between classes*.
<!--
BEGIN QUESTION
name: q7aii
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

---
## Question 8.

<!-- BEGIN QUESTION -->
### Question 8a

How would you describe Northpointe’s definition of fairness, after learning and utilizing the metrics they used? How is this different from your description of ProPublica’s definition from Q5aii? 
<!--
BEGIN QUESTION
name: q8a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 8b 

Why might Northpointe be more interested in these specific metrics of fairness and accuracy (both the AUC and the deliberate neglect of the race feature), given trends of colorblindness in the US?
<!--
BEGIN QUESTION
name: q8b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 8c

If anything, what are ProPublica and Northpointe each not considering in their definitions? (Hint: Think about your knowledge of the historical context of policing data.)
<!--
BEGIN QUESTION
name: q8c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

So far, we’ve investigated ProPublica’s and Northpointe’s metrics and definitions of fairness. In the world of machine learning there are [many more](https://fairmlbook.org/tutorial2.html), so in the next section we will take a look at a third definition.

----

# Part 4. Yet Another Definition of Fairness <a id="part-four"></a>

In this section, you will go through yet another metric used to evaluate fairness in machine learning: disparate impact. Disparate impact is a legal doctrine that determines if there is unintended discrimination towards a protected class ([Society for Human Resource Management](https://www.shrm.org/resourcesandtools/tools-and-samples/hr-qa/pages/disparateimpactdisparatetreatment.aspx)). In machine learning, disparate impact is a metric to evaluate fairness in a model. It is a form of bias within an algorithm that reflects systemic discrimination when a model’s outputs are dependent on a ***sensitive feature*** (the protected class). This is often considered unintentional (like the legal doctrine) due to the fact that the sensitive feature is omitted from the model, though it is still correlated with the output through proxy variables ([Wang et al.](https://arxiv.org/pdf/1801.05398.pdf#:~:text=Abstract%E2%80%94In%20the%20context%20of,e.g.%2C%20race%20or%20gender)).

Instead of simply evaluating the fairness of the COMPAS tool after the fact, we will notice and remove bias from the dataset to actually change the outputs of the model, and therefore demonstrate another technique aimed at creating a fair algorithm. In order to do so, we need to quantify the disparate impact in the dataset and use aif360 (a special Python package with fairness tools for machine learning) to remove the bias completely. 

----
## Question 9. Disparate Impact: Quantification and Removal

First, let’s visualize the disparity that we would like to remove from the dataset by utilizing the same histogram we created in question 3.  

In [ ]:
unpriv = select_data[select_data["race"] == "Caucasian"]
priv = select_data[select_data["race"] == "African-American"]
sns.distplot(priv["decile_score"], hist=True, rug=False)
sns.distplot(unpriv["decile_score"], hist=True, rug=False);

As you observed in question 3, this histogram illuminates the disparity in scores for Black and white defendants. 

Now, let’s quantify the disparate impact we are seeing in the plot. In order to do that we need to distinguish between a **privileged group** and an **unprivileged group**. In technical terms, *the privileged group receives disproportionately higher scores from a trained model than the unprivileged group*, so therefore the Black defendants will be considered "privileged" and the white defendants will be considered "unprivileged" in this case. In machine learning, we can understand disparate impact as the proportion of individuals that get positive outcomes (positive outcomes, meaning if they received a high score) for the two groups described above.

The function below calculates disparate impact exactly as described above. We then apply this function to our own dataset to get the following quantification.

In [ ]:
def proportion(data, group):
    # Returns the proportion of individuals in data from the group who get a high risk score
    race_group = data[data["race"] == group]
    positive_outcomes = race_group[race_group["decile_score"] > 4]
    return len(positive_outcomes) / len(race_group)

In [ ]:
prob_priv = proportion(select_data, "African-American")
prob_unpriv = proportion(select_data, "Caucasian")
prob_unpriv / prob_priv

In industry, if the proportion of unprivileged individuals receiving positive outcomes to privileged individuals receiving positive outcomes is less than 80%, there is a disparate impact violation.

<!-- BEGIN QUESTION -->
### Question 9a
Is there a disparate impact violation in our dataset? What is one reason that could explain this outcome? 
<!--
BEGIN QUESTION
name: q9a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

In order to stop a trained model from replicating these biases in its output, we can now use the Python package aif360’s Disparate Impact Remover (a tool that was created and designed specifically to eliminate disparate impact in a dataset) to remove the bias we just calculated. The code below does this by creating a specific type of dataset ([documentation for BinaryLabelDataset](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.BinaryLabelDataset.html#aif360.datasets.BinaryLabelDataset)) from our original dataset and creating a DisparateImpactRemover type object ([documentation for DisparateImpactRemover](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.DisparateImpactRemover.html)) with the new dataset that eliminates the disparate impact and outputs a transformed dataset. You do not need to understand this code, but feel free to go through it yourself!



In [ ]:
# Create new DataFrame with just the necessary columns - Only numeric values for race, decile score, and two year recid
race_data = select_data[(select_data["race"] == "Caucasian") | (select_data["race"] == "African-American")]
race_col = pd.get_dummies(race_data, "race")["race_Caucasian"]
noise = np.random.normal(0, 0.1, race_data.shape[0])
decile_col = race_data["decile_score"] + noise
recid_col = race_data["two_year_recid"]
new_df = pd.DataFrame({"race": race_col, "decile_score": decile_col, "two_year_recid": recid_col})

# Create BinaryLabelDataset
BLD = BinaryLabelDataset(favorable_label=1, # Positive Outcome
                         unfavorable_label=0, # Negative Outcome
                         df=new_df,
                         label_names=["two_year_recid"],
                         protected_attribute_names=["race"],
                         unprivileged_protected_attributes=[1])

In [ ]:
remover = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="race")
transformed_data = remover.fit_transform(BLD)

### Question 9b
Using the code from our original histogram, create your own histogram to plot the scores from the modified dataset (transformed_df, created below).

In [ ]:
# Transform output from DIRemover into usable DataFrame
# Use this dataset instead of select_data
transformed_df = pd.DataFrame(np.hstack([transformed_data.features, transformed_data.labels]),
                              columns=["race","decile_score","two_year_recid"])

In [ ]:
unpriv_t = ...[...["race"] == 1]
priv_t = ...[...["race"] == 0]
sns.distplot(...["decile_score"], hist=True, rug=False)
sns.distplot(...["decile_score"], hist=True, rug=False);

### Question 9c 
Using code from above, once again calculate the disparate impact, but this time on the modified dataset (transformed_df).

In [ ]:
# Calculate disparate impact
# Use transformed_df instead of select_data
prob_priv_t = proportion(..., ...)
prob_unpriv_t = proportion(..., ...)
... / ...

<!-- BEGIN QUESTION -->
### Question 9d

What has changed from our original histogram? Please explain why this change has happened. 
<!--
BEGIN QUESTION
name: q9d
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 9e

How would you describe this third definition of fairness, after learning and utilizing these new metrics?
<!--
BEGIN QUESTION
name: q9e
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 9f 

How does this definition of fairness differ from ProPublica’s and Northpointe’s? 
<!--
BEGIN QUESTION
name: q9f
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

---

## Question 10: Considering Expertise Outside of Data Science

Just now, we have used technical data science skills to computationally remove bias from the data set. By removing bias, we’ve made the outputs of the algorithm statistically fair in regards to one definition of fairness. However, it is important to consider many types of knowledge and experiences beyond data science expertise when analyzing and creating an algorithm like COMPAS. As such, you will think through issues of expertise and fairness in the next set of questions.

<!-- BEGIN QUESTION -->
### Question 10a

Look back to your answer from Q0a. Now that you’ve gone through several definitions of fairness, how would you add to or revise your answer to: Explain 3 parties that are impacted by the COMPAS tool. In what ways are they impacted?
<!--
BEGIN QUESTION
name: q10a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 10b

What expertise and lived experiences are necessary to understand and critically think about the issues produced by COMPAS?
<!--
BEGIN QUESTION
name: q10b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 10c

Why is this third definition of fairness still inadequate as a measurement of justice in the court system? (Hint: look at the previous two questions and answers).
<!--
BEGIN QUESTION
name: q10c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

------

# Part 5. Conclusion <a id="part-five"></a>

## Question 11: Which Definition is Fair? And Who Decides?

We’ve now gone through three definitions of fairness, each one with a different idea of how to operationalize fairness and to judge whether or not an algorithm is fair. As a data scientist, you may encounter situations where you will need to make decisions that affect real-world outcomes and people! Let’s try to do this for COMPAS. 

<!-- BEGIN QUESTION -->
### Question 11a

If you had to decide between the three definitions of fairness above, which definition do you think would make “fair” decisions for everyone who goes through the court system? What values did you consider as you made this decision? If you cannot come to a decision, what challenges did you come across when considering this? 
<!--
BEGIN QUESTION
name: q11a
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->
### Question 11b

Take a step back and think about how different actors who created, utilize, and are affected by COMPAS would consider which definition is most fair. Name two relevant actors, and discuss what they would value in their own definitions of fairness. Of the three definitions you have explored, which would they decide is most fair from the perspective of that actor? If you don’t think they’d choose any of them, explain why. (Examples of actors, which you’re welcome to use: judges, defendants, police, policy makers, community members) 
<!--
BEGIN QUESTION
name: q11b
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

Choosing one definition of fairness can be incredibly difficult when you need to consider all the actors at play. Throughout this module we have examined where and how the COMPAS algorithm is appropriate to use. It is also important to recognize the problems that are not solvable by an algorithm and think through how we can make the ecosystem that COMPAS is in  (which includes but is not limited to the legal system, affected communities, the tech industry, etc.) more just.

<!-- BEGIN QUESTION -->
### Question 11c

What issues that are relevant to the COMPAS ecosystem but outside of the algorithm itself need to be addressed to be able to create a more just system, with or without the algorithm? 
<!--
BEGIN QUESTION
name: q11c
points: 1
manual: true
-->

**Answer:** *YOUR ANSWER HERE*

<!-- END QUESTION -->

You’ve now begun to think through the very complex systems in which the COMPAS algorithm functions. Congratulations! Through considering a few of the differing definitions of fairness connected to COMPAS, hopefully you can begin to understand some of the human contexts of  creating algorithms that intentionally affect people and their decision-making. 

----

In [ ]:
# Save your notebook first, then go to `Kernel` > `Restart and Run All Cells`
# Download the zip file, which contains a copy of your notebook and your written responses.
generator.export("notebook4.ipynb")